<a href="https://colab.research.google.com/github/GenAIHub/genai-workshop/blob/main/03_Agents/02_simple_multistep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Step Query Engine

We have a multi-step query engine that's able to decompose a complex query into sequential subquestions. This
guide walks you through how to set it up!

In [ ]:
!pip install boto3
!pip install llama-index
!pip install llama-index-llms-bedrock
!pip install llama-index-embeddings-bedrock

#### Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

#### Load documents, build the VectorStoreIndex

In [ ]:
import os
import boto3

# Set AWS env config
region_name = os.getenv("AWS_REGION", "us-east-1")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID", "")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY", "")

In [ ]:
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import Settings

llm_model_id = "anthropic.claude-3-haiku-20240307-v1:0"
embed_model_id = "amazon.titan-embed-text-v1"

llm = Bedrock(
    model=llm_model_id,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
    temperature=0.1,
    max_tokens=512
    )

embed_model = BedrockEmbedding(
    model=embed_model_id,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
    )

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [ ]:
index = VectorStoreIndex.from_documents(documents)

#### Query Index

In [ ]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

single_step_query_engine = index.as_query_engine(llm=llm, similarity_top_k=4)
step_decompose_transform = StepDecomposeQueryTransform(llm=llm, verbose=True)

In [ ]:
index_summary = "Ask me anything."

In [ ]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine


query_engine = MultiStepQueryEngine(
    query_engine=single_step_query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
    num_steps=6
)
response = query_engine.query(
    "What were Paul Graham's achievements?",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

In [ ]:
sub_qa = response.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)